In [1]:
import re
from math import log
from functools import total_ordering

@total_ordering
class TQuality:        
    def __init__(self, del_prob, avg_latency, ovh_ratio, identifier=""):
        self.p =   int(del_prob * 20)
        self.l = - int(log(avg_latency) / log(2))
        self.o = - int(ovh_ratio)
        self.name = identifier
    
    # Method for comparing two transmission quality descriptors.
    # 
    # @otherTQ: Transmission quality descriptor to compare with.
    # @return:  Positive, if this object describes better transmission quality;
    #           Negative, if this object describes worse transmission quality;
    #           Zero, if both objects correspond to equivalent transmission quality.
    def compareTo(self, otherTQ):
        if (self.p > otherTQ.p):
            return 1;
        elif (self.p == otherTQ.p and self.l > otherTQ.l):
            return 1;
        elif (self.p == otherTQ.p and self.l == otherTQ.l and self.o > otherTQ.o):
            return 1;
        elif (self.p == otherTQ.p and self.l == otherTQ.l and self.o == otherTQ.o):
            return 0;
        else:
            return -1;
        
    def __gt__(self, otherTQ):
        return self.compareTo(otherTQ) > 0
        
    def __eq__(self, otherTQ):
        return self.compareTo(otherTQ) == 0
    
    def simulation_parameters(self):
        params = re.split("-|_", self.name)
        return params[2] + "|" + params[3] + "|" + params[4] + "|" + params[5]
    
    def quality_metrics(self):
        return str(self.p) + "|" + str(self.l) + "|" + str(self.o)

In [2]:
from pprint import pprint

a = TQuality(1,1,1)
pprint(vars(a))
print a.compareTo(TQuality(1,0.1,1))

{'l': 0, 'name': '', 'o': -1, 'p': 20}
-1


In [3]:
import glob

rel_file_paths = glob.glob("final_comparison/*.txt")

In [4]:
import re

sample_path = rel_file_paths[0]

print re.split("-|_", sample_path)

['final', 'comparison/finalComparison', 'EpidemicOracleRouter', '2048', '2048M', '128k', 'MessageStatsReport.txt']


## TODO
* We want to sort the scenarios according to transmission quality.
    * Find a way for reading quality information from the MessageStatsReport's
    * Fill a vector with quality information for all tested scenarios
    * Sort that vector using compareTo

In [5]:
def getTransmissionQualityFromReport(report_path):
    with open(report_path) as f:
        for line in f:
            split = line.split()
            if "delivery_prob" in line:
                p = float(split[1])
            elif "latency_avg" in line:
                if "NaN" in line:
                    l = 1
                else:
                    l = float(split[1])
            elif "overhead_ratio" in line:
                if "NaN" in line:
                    o = 1
                else:
                    o = float(split[1])

    return TQuality(p, l, o, report_path)

In [6]:
b = getTransmissionQualityFromReport(sample_path)
pprint(vars(b))

{'l': -5,
 'name': 'final_comparison/finalComparison-EpidemicOracleRouter-2048-2048M-128k_MessageStatsReport.txt',
 'o': -3005,
 'p': 20}


In [7]:
for report_path in rel_file_paths:
    pprint(vars(getTransmissionQualityFromReport(report_path)))

{'l': -5,
 'name': 'final_comparison/finalComparison-EpidemicOracleRouter-2048-2048M-128k_MessageStatsReport.txt',
 'o': -3005,
 'p': 20}
{'l': -10,
 'name': 'final_comparison/finalComparison-EpidemicOracleRouter-32-64M-1M_MessageStatsReport.txt',
 'o': -29,
 'p': 7}
{'l': -9,
 'name': 'final_comparison/finalComparison-EpidemicOracleRouter-256-1M-128k_MessageStatsReport.txt',
 'o': -376,
 'p': 2}
{'l': -7,
 'name': 'final_comparison/finalComparison-EpidemicRouter-2048-64M-1M_MessageStatsReport.txt',
 'o': -2843,
 'p': 18}
{'l': -9,
 'name': 'final_comparison/finalComparison-DirectDeliveryRouter-2048-64M-128k_MessageStatsReport.txt',
 'o': 0,
 'p': 2}
{'l': 0,
 'name': 'final_comparison/finalComparison-PassiveRouter-32-2048M-1M_MessageStatsReport.txt',
 'o': -1,
 'p': 0}
{'l': -10,
 'name': 'final_comparison/finalComparison-SprayAndWaitRouter-2048-64M-128k_MessageStatsReport.txt',
 'o': -19,
 'p': 5}
{'l': -8,
 'name': 'final_comparison/finalComparison-EpidemicOracleRouter-256-64M-128k_

In [8]:
TQualityArray = map(getTransmissionQualityFromReport, rel_file_paths)
sortedTQualityArray = sorted(TQualityArray, reverse=True)

In [9]:
map(lambda q: vars(q), TQualityArray)

[{'l': -5,
  'name': 'final_comparison/finalComparison-EpidemicOracleRouter-2048-2048M-128k_MessageStatsReport.txt',
  'o': -3005,
  'p': 20},
 {'l': -10,
  'name': 'final_comparison/finalComparison-EpidemicOracleRouter-32-64M-1M_MessageStatsReport.txt',
  'o': -29,
  'p': 7},
 {'l': -9,
  'name': 'final_comparison/finalComparison-EpidemicOracleRouter-256-1M-128k_MessageStatsReport.txt',
  'o': -376,
  'p': 2},
 {'l': -7,
  'name': 'final_comparison/finalComparison-EpidemicRouter-2048-64M-1M_MessageStatsReport.txt',
  'o': -2843,
  'p': 18},
 {'l': -9,
  'name': 'final_comparison/finalComparison-DirectDeliveryRouter-2048-64M-128k_MessageStatsReport.txt',
  'o': 0,
  'p': 2},
 {'l': 0,
  'name': 'final_comparison/finalComparison-PassiveRouter-32-2048M-1M_MessageStatsReport.txt',
  'o': -1,
  'p': 0},
 {'l': -10,
  'name': 'final_comparison/finalComparison-SprayAndWaitRouter-2048-64M-128k_MessageStatsReport.txt',
  'o': -19,
  'p': 5},
 {'l': -8,
  'name': 'final_comparison/finalComparis

In [10]:
map(lambda q: vars(q), sortedTQualityArray)

[{'l': -5,
  'name': 'final_comparison/finalComparison-EpidemicOracleRouter-2048-2048M-128k_MessageStatsReport.txt',
  'o': -3005,
  'p': 20},
 {'l': -5,
  'name': 'final_comparison/finalComparison-EpidemicOracleRouter-2048-2048M-1M_MessageStatsReport.txt',
  'o': -3005,
  'p': 20},
 {'l': -5,
  'name': 'final_comparison/finalComparison-EpidemicOracleRouter-2048-64M-128k_MessageStatsReport.txt',
  'o': -37272,
  'p': 20},
 {'l': -5,
  'name': 'final_comparison/finalComparison-EpidemicOracleRouter-2048-64M-1M_MessageStatsReport.txt',
  'o': -37272,
  'p': 20},
 {'l': -7,
  'name': 'final_comparison/finalComparison-EpidemicRouter-2048-2048M-1M_MessageStatsReport.txt',
  'o': -2161,
  'p': 18},
 {'l': -7,
  'name': 'final_comparison/finalComparison-EpidemicRouter-2048-64M-1M_MessageStatsReport.txt',
  'o': -2843,
  'p': 18},
 {'l': -8,
  'name': 'final_comparison/finalComparison-EpidemicRouter-256-2048M-1M_MessageStatsReport.txt',
  'o': -252,
  'p': 16},
 {'l': -8,
  'name': 'final_compa

In [11]:
sorted_report_paths = map(lambda q: q.name, sortedTQualityArray)

In [12]:
map(lambda q: q.simulation_parameters() + "|" + q.quality_metrics(), sortedTQualityArray)

['EpidemicOracleRouter|2048|2048M|128k|20|-5|-3005',
 'EpidemicOracleRouter|2048|2048M|1M|20|-5|-3005',
 'EpidemicOracleRouter|2048|64M|128k|20|-5|-37272',
 'EpidemicOracleRouter|2048|64M|1M|20|-5|-37272',
 'EpidemicRouter|2048|2048M|1M|18|-7|-2161',
 'EpidemicRouter|2048|64M|1M|18|-7|-2843',
 'EpidemicRouter|256|2048M|1M|16|-8|-252',
 'EpidemicRouter|256|64M|1M|16|-8|-253',
 'EpidemicOracleRouter|256|2048M|128k|16|-8|-353',
 'EpidemicOracleRouter|256|2048M|1M|16|-8|-353',
 'EpidemicOracleRouter|256|64M|128k|16|-8|-363',
 'EpidemicOracleRouter|256|64M|1M|16|-8|-363',
 'EpidemicRouter|2048|2048M|128k|14|-9|-2042',
 'EpidemicRouter|2048|64M|128k|14|-9|-2042',
 'SprayAndWaitRouter|256|64M|1M|10|-9|-9',
 'SprayAndWaitRouter|256|2048M|1M|10|-9|-9',
 'EpidemicRouter|256|64M|128k|9|-9|-174',
 'EpidemicRouter|256|2048M|128k|9|-9|-174',
 'SprayAndWaitRouter|256|1M|1M|7|-9|-13',
 'EpidemicOracleRouter|32|64M|1M|7|-10|-29',
 'EpidemicOracleRouter|32|2048M|1M|7|-10|-29',
 'EpidemicOracleRouter|32|

In [13]:
def print_file(file_path):
    with open(file_path) as f:
        for line in f:
            print line,